In [55]:
import numpy as np
import pandas as pd
import io 

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

Exploracion de datos

In [56]:
df_test = pd.read_csv("test.txt")
df_train = pd.read_csv("train.txt")

Base para pruebas

In [57]:
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


Base para entranamiento

In [21]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [24]:
df_train.shape,


((891, 12),)

In [25]:
df_test.shape

(418, 11)

In [58]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [60]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


Conteo de valores nulos

In [61]:
pd.isnull(df_train).sum()
pd.isnull(df_test).sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [62]:
df_train.describe()
df_test.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [66]:
df_train['Sex'].replace(['female','male'],[0,1], inplace= True)
df_test ['Sex'].replace(['female','male'],[0,1], inplace= True)

In [65]:
df_train['Embarked'].replace(['Q','S', 'C'],[0,1,2], inplace= True)
df_test ['Embarked'].replace(['Q','S', 'C'],[0,1,2], inplace= True)

In [67]:
df_train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,1.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,2.0
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,1.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,1.0
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",1,27.0,0,0,211536,13.0000,NaN,1.0
887,888,1,1,"Graham, Miss. Margaret Edith",0,19.0,0,0,112053,30.0000,B42,1.0
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",0,NaN,1,2,W./C. 6607,23.4500,NaN,1.0
889,890,1,1,"Behr, Mr. Karl Howell",1,26.0,0,0,111369,30.0000,C148,2.0


In [33]:
df_test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",1,34.5,0,0,330911,7.8292,NaN,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",0,47.0,1,0,363272,7.0000,NaN,1
2,894,2,"Myles, Mr. Thomas Francis",1,62.0,0,0,240276,9.6875,NaN,0
3,895,3,"Wirz, Mr. Albert",1,27.0,0,0,315154,8.6625,NaN,1
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",0,22.0,1,1,3101298,12.2875,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",1,NaN,0,0,A.5. 3236,8.0500,NaN,1
414,1306,1,"Oliva y Ocana, Dona. Fermina",0,39.0,0,0,PC 17758,108.9000,C105,2
415,1307,3,"Saether, Mr. Simon Sivertsen",1,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,1
416,1308,3,"Ware, Mr. Frederick",1,NaN,0,0,359309,8.0500,NaN,1


Reemplazo del promedio en el campo Fare

In [69]:
df_test['Fare'].mean()
promreemplazo = 35.63
df_test['Fare'] = df_train['Fare'].replace(np.nan, promreemplazo)


Imputacion de la mediana

In [70]:
df_train['Age'].median()
df_test['Age'].median()
mediana = 27
df_train['Age'] = df_train['Age'].replace(np.nan, mediana)
df_test['Age'] = df_train['Age'].replace(np.nan, mediana)

Eliminacion de Variable Cabin

In [71]:
df_train.drop(['Cabin'],axis=1 , inplace=True)
df_test.drop(['Cabin'],axis=1 , inplace=True)

In [72]:
df_train = df_train.drop(['PassengerId', 'Name','Ticket'], axis=1)
df_test = df_test.drop(['Name', 'Ticket'], axis=1)

In [73]:
df_train.dropna(axis=0, how='any', inplace=True)

In [74]:
pd.isnull(df_train).sum()
pd.isnull(df_test).sum()

PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
dtype: int64

In [75]:
X = df_train.iloc[:, df_train.columns != 'Survived']
y = df_train.loc[:, 'Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True)

In [76]:
X_train.shape[1]
inputs = X_train.shape[1]

In [77]:
ids = df_test ['PassengerId']

Primera Arquitectura

In [87]:
def red_v1():
  #base secuencial para definir la estructura de la Red 
  clasificador = Sequential()

  #primera capa oclta
  clasificador.add(Dense(input_dim = 7, units=5, activation='relu', kernel_initializer='uniform'))

  #segunda capa oculta
  clasificador.add(Dense(units=4, activation='relu', kernel_initializer='uniform'))

  #capa de salida
  clasificador.add(Dense(units=1, activation='sigmoid', kernel_initializer='uniform'))

  #parametros de optimización
  clasificador.compile(optimizer='SGD', loss='binary_crossentropy', metrics=['accuracy'])

  return clasificador

In [88]:
clasificador = KerasClassifier(build_fn=red_v1, batch_size=25, nb_epoch=500)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  """Entry point for launching an IPython kernel.


In [89]:
kfolds = KFold(n_splits=5, shuffle=True)

accs = cross_val_score(clasificador, X=X_train, y=y_train, cv=kfolds, verbose=1, n_jobs=-1)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    3.2s finished


In [90]:
accs

array([0.62400001, 0.68000001, 0.58064514, 0.58064514, 0.58064514])

In [91]:
accs.mean()

0.6091870903968811

Segunda Arquitectura

In [83]:
def red_v2():
  #base secuencial para definir la estructura de la Red 
  clasificador = Sequential()

  #primera capa oclta
  clasificador.add(Dense(input_dim = 7, units=3, activation='relu', kernel_initializer='uniform'))

  #segunda capa oculta
  clasificador.add(Dense(units=4, activation='relu', kernel_initializer='uniform'))

  #capa de salida
  clasificador.add(Dense(units=1, activation='sigmoid', kernel_initializer='uniform'))

  #parametros de optimización
  clasificador.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

  return clasificador


In [92]:
clasificador = KerasClassifier(build_fn=red_v2, batch_size=35, nb_epoch=500)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  """Entry point for launching an IPython kernel.


In [93]:
kfolds = KFold(n_splits=5, shuffle=True)

accs2 = cross_val_score(clasificador, X=X_train, y=y_train, cv=kfolds, verbose=1, n_jobs=-1)
accs2.mean()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    3.8s finished


0.6156516075134277

Tercera arquitectura

In [94]:
def red_v3():
  #base secuencial para definir la estructura de la Red 
  clasificador = Sequential()

  #primera capa oclta
  clasificador.add(Dense(input_dim = 7, units=8, activation='relu', kernel_initializer='uniform'))

  #segunda capa oculta
  clasificador.add(Dense(units=8, activation='relu', kernel_initializer='uniform'))

  #capa de salida
  clasificador.add(Dense(units=1, activation='sigmoid', kernel_initializer='uniform'))

  #parametros de optimización
  clasificador.compile(optimizer='RMSprop', loss='binary_crossentropy', metrics=['accuracy'])

  return clasificador

  clasificador = KerasClassifier(build_fn=red_v3, batch_size=36, nb_epoch=2500)
  kfolds = KFold(n_splits=10, shuffle=True)

accs3 = cross_val_score(clasificador, X=X_train, y=y_train, cv=kfolds, verbose=1, n_jobs=-1)
accs3.mean()


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    3.3s finished


0.6029935479164124

Cuarta Arquitectura

In [95]:
def red_v4():
  #base secuencial para definir la estructura de la Red 
  clasificador = Sequential()

  #primera capa oclta
  clasificador.add(Dense(input_dim = 7, units=3, activation='relu', kernel_initializer='uniform'))

  #segunda capa oculta
  clasificador.add(Dense(units=10, activation='relu', kernel_initializer='uniform'))

  #capa de salida
  clasificador.add(Dense(units=1, activation='sigmoid', kernel_initializer='uniform'))

  #parametros de optimización
  clasificador.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

  return clasificador

  clasificador = KerasClassifier(build_fn=red_v4, batch_size=40, nb_epoch=5000)
  kfolds = KFold(n_splits=10, shuffle=True)

accs4 = cross_val_score(clasificador, X=X_train, y=y_train, cv=kfolds, verbose=1, n_jobs=-1)
accs4.mean()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    4.7s finished


0.6093290328979493

Quinta arquitectura

In [96]:
def red_v5():
  #base secuencial para definir la estructura de la Red 
  clasificador = Sequential()

  #primera capa oclta
  clasificador.add(Dense(input_dim = 7, units=2, activation='relu', kernel_initializer='uniform'))

  #segunda capa oculta
  clasificador.add(Dense(units=4, activation='relu', kernel_initializer='uniform'))

  #capa de salida
  clasificador.add(Dense(units=1, activation='sigmoid', kernel_initializer='uniform'))

  #parametros de optimización
  clasificador.compile(optimizer='SGD', loss='binary_crossentropy', metrics=['accuracy'])

  return clasificador

  clasificador = KerasClassifier(build_fn=red_v5, batch_size=50, nb_epoch=2500)
  kfolds = KFold(n_splits=10, shuffle=True)

accs5 = cross_val_score(clasificador, X=X_train, y=y_train, cv=kfolds, verbose=1, n_jobs=-1)
accs5.mean()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    3.6s finished


0.5996516108512878